In [1]:
!pip install optuna

In [211]:
from utils import *
from sklearn.tree import DecisionTreeClassifier
import pandas as pd
import numpy as np
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.impute import SimpleImputer
import xgboost as xgb
import optuna
from optuna import TrialPruned
from utils import *
from sklearn.model_selection import train_test_split, RandomizedSearchCV, cross_val_score, StratifiedKFold
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import pickle
import json
from sklearn.linear_model import LassoCV

In [212]:
df_Stress = pd.read_excel("Copia de Behavioral-features - per minute.xlsx")

In [213]:
df_Stress.head(10)

,PP,Blok,Condition,timestamp,Valence_rc,Arousal_rc,Dominance,Stress,MentalEffort,MentalDemand,...,WristLeft_HandLeftPlaneYZAxisZstdv,ShoulderRight_ElbowRightPlaneZXAxisXstdv,ShoulderRight_ElbowRightPlaneXYAxisYstdv,ShoulderRight_ElbowRightPlaneYZAxisZstdv,ElbowRight_WristRightPlaneZXAxisXstdv,ElbowRight_WristRightPlaneXYAxisYstdv,ElbowRight_WristRightPlaneYZAxisZstdv,WristRight_HandRightPlaneZXAxisXstdv,WristRight_HandRightPlaneXYAxisYstdv,WristRight_HandRightKinectZAxisstdv
0,PP1,1,R,20120918T131600000,NaN,NaN,NaN,NaN,NaN,NaN,...,166.872086,22.946337,5.605230,13.237048,30.932939,102.460262,8.444481,54.436879,75.886750,155.734343
1,PP1,1,R,20120918T131700000,NaN,NaN,NaN,NaN,NaN,NaN,...,58.237714,7.194860,2.468881,3.886001,2.000359,3.431337,1.160298,4.579341,2.868843,171.486911
2,PP1,1,R,20120918T131800000,NaN,NaN,NaN,NaN,NaN,NaN,...,64.548824,14.726365,2.953021,9.185623,13.379140,48.162012,30.508877,23.013755,30.629061,149.748809
3,PP1,1,R,20120918T131900000,NaN,NaN,NaN,NaN,NaN,NaN,...,38.209895,2.542686,1.564318,2.453674,4.796273,8.223865,1.114906,3.854074,4.048611,173.149430
4,PP1,1,R,20120918T132000000,NaN,NaN,NaN,NaN,NaN,NaN,...,80.171422,9.905241,20.342488,5.552234,7.198096,22.942488,46.386278,17.514534,29.702675,165.368301
5,PP1,1,R,20120918T132100000,NaN,NaN,NaN,NaN,NaN,NaN,...,98.907746,7.748967,1.058581,5.632123,6.953512,5.591442,3.735698,30.137148,57.577431,151.554940
6,PP1,1,N,20120918T132600000,6.0,4.0,6.0,4.1,5.1,0.7,...,81.208417,2.193391,2.353367,1.800807,3.298600,5.711348,65.979538,26.049338,123.564767,36.918869
7,PP1,1,N,20120918T132700000,6.0,4.0,6.0,4.1,5.1,0.7,...,155.183912,9.271625,8.185302,4.089097,6.906731,9.327650,76.878457,28.215609,107.462005,105.295347
8,PP1,1,N,20120918T132800000,6.0,4.0,6.0,4.1,5.1,0.7,...,53.912808,19.278932,2.330069,8.432293,19.630151,58.211049,98.285955,23.155413,99.189884,161.392789
9,PP1,1,N,20120918T132900000,6.0,4.0,6.0,4.1,5.1,0.7,...,21.500961,4.079656,1.618863,0.916299,4.096824,30.095558,149.907085,24.848498,89.450385,142.564028


MISMO CÓDIGO APLICANDO LASSO

In [216]:


# Hago una copia del dataframe para no modificar el original
df_stress = df_Stress.copy()

# Eliminamos columnas innecesarias
columnas_a_eliminar = ['PP','timestamp','HR','RMSSD', 'RMSSD','SCL','Valence_rc','Arousal_rc',
                       'Dominance','MentalEffort','MentalDemand','PhysicalDemand','TemporalDemand',
                       'Effort','Performance_rc','Frustration','NasaTLX','Squality','Sneutral','Shappy',
                       'Ssad','Sangry','Ssurprised','Sscared','Sdisgusted','Svalence']
df_stress = df_stress.drop(columnas_a_eliminar, axis=1)

# Filtramos filas con condición diferente a 'R'
df_stress = df_stress[df_stress['Condition'] != 'R']

# Convertimos la variable objetivo a int
df_stress['Stress'] = df_stress['Stress'].astype(int)

# Dividimos los datos
X = df_stress.drop('Stress', axis=1)
y = df_stress['Stress']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=True, random_state=42, stratify=df_stress['Condition'])

# Variables categóricas
categorical_features = ['Condition', 'Blok']

# Convertimos en categóricas
X_train[categorical_features] = X_train[categorical_features].astype('category')
X_train = pd.get_dummies(X_train, columns=categorical_features)

# Variables numéricas
numeric_features = X_train.select_dtypes(include=[np.number]).columns.tolist()

# Imputación de valores nulos
imputer = SimpleImputer(strategy='most_frequent')
imputer.fit(X_train[numeric_features])
X_train[numeric_features] = imputer.transform(X_train[numeric_features])

# Normalización
scaler = StandardScaler()
X_train[numeric_features] = scaler.fit_transform(X_train[numeric_features])

# Aplicamos Lasso para selección de características
lasso = LassoCV(cv=5, random_state=42, max_iter=5000)  # Aumentar iteraciones
lasso.fit(X_train, y_train)

# Seleccionamos solo las características con coeficientes distintos de cero
selected_features = X_train.columns[lasso.coef_ != 0]
X_train = X_train[selected_features]

print(f"Seleccionadas {len(selected_features)} características tras regularización Lasso.")

# Aplicamos las mismas transformaciones a X_test
X_test[categorical_features] = X_test[categorical_features].astype('category')
X_test = pd.get_dummies(X_test, columns=categorical_features)
X_test[numeric_features] = imputer.transform(X_test[numeric_features])
X_test[numeric_features] = scaler.transform(X_test[numeric_features])
X_test = X_test[selected_features]


Seleccionadas 136 características tras regularización Lasso.


In [217]:
def objective(trial):
    # Parámetros optimizados con rangos ajustados según mejores valores encontrados
    params = {
        #'criterion': 'gini',
        #'splitter': 'best',
        'max_depth':trial.suggest_int('max_depth', 2,20),
        'min_samples_leaf': trial.suggest_int('min_samples_leaf', 2, 4),
        #'min_weight_fraction_leaf': trial.suggest_float('min_weight_fraction_leaf', 0.01, 0.05, step=0.01),
        #'max_features': trial.suggest_categorical('max_features', ['sqrt', 'log2']),
        'max_leaf_nodes': trial.suggest_int('max_leaf_nodes', 100, 300),
        'min_impurity_decrease': trial.suggest_float('min_impurity_decrease', 0, 0.2, step=0.001),
        #'class_weight': trial.suggest_categorical('class_weight', ['balanced', 'balanced_subsample']),
        #'ccp_alpha': trial.suggest_float('ccp_alpha', 0.001, 0.002, step=0.001),
        #'random_state': 42,
    
    }

    # Modelo
    model = DecisionTreeClassifier(**params)

    # Validación cruzada con StratifiedKFold
    cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

    try:
        scores = []
        for fold, (train_idx, valid_idx) in enumerate(cv.split(X_train, y_train)):
            X_t, X_v = X_train.iloc[train_idx], X_train.iloc[valid_idx]
            y_t, y_v = y_train.iloc[train_idx], y_train.iloc[valid_idx]

            model.fit(X_t, y_t)
            val_score = model.score(X_v, y_v)
            scores.append(val_score)

            # Reportar progreso a Optuna
            trial.report(val_score, fold)

            # Aplicar pruning si el rendimiento es bajo
            if trial.should_prune():
                raise TrialPruned()

        return np.mean(scores)
    except Exception as e:
        print(f"Error con los parámetros: {params}, Error: {e}")
        return float("nan")  # Devuelve NaN para que Optuna ignore esta combinación

In [218]:
# Crear estudio
study = optuna.create_study(direction='maximize')

# Optimización
study.optimize(objective, n_trials=20)

# Mejor resultado encontrado
print("Mejores parámetros:", study.best_params)

[I 2025-03-26 21:58:26,169] A new study created in memory with name: no-name-1efb90ee-f50f-4bec-b25e-d1c42e8c3763
[I 2025-03-26 21:58:26,362] Trial 0 finished with value: 0.246046511627907 and parameters: {'max_depth': 10, 'min_samples_leaf': 3, 'max_leaf_nodes': 201, 'min_impurity_decrease': 0.045}. Best is trial 0 with value: 0.246046511627907.
[I 2025-03-26 21:58:26,546] Trial 1 finished with value: 0.246046511627907 and parameters: {'max_depth': 20, 'min_samples_leaf': 4, 'max_leaf_nodes': 185, 'min_impurity_decrease': 0.043000000000000003}. Best is trial 0 with value: 0.246046511627907.
[I 2025-03-26 21:58:26,740] Trial 2 finished with value: 0.246046511627907 and parameters: {'max_depth': 14, 'min_samples_leaf': 2, 'max_leaf_nodes': 202, 'min_impurity_decrease': 0.116}. Best is trial 0 with value: 0.246046511627907.
[I 2025-03-26 21:58:27,375] Trial 3 finished with value: 0.4674418604651163 and parameters: {'max_depth': 15, 'min_samples_leaf': 4, 'max_leaf_nodes': 144, 'min_impur

Mejores parámetros: {'max_depth': 15, 'min_samples_leaf': 3, 'max_leaf_nodes': 122, 'min_impurity_decrease': 0.003}


In [219]:
best_value = study.best_value
print(f"Mejor accuracy: {best_value}")

Mejor accuracy: 0.7130232558139534


In [220]:
study.best_params

{'max_depth': 15,
 'min_samples_leaf': 3,
 'max_leaf_nodes': 122,
 'min_impurity_decrease': 0.003}

In [223]:
model = DecisionTreeClassifier(**study.best_params)
model.fit(X_train, y_train)

# Predicciones en train y test
y_train_pred = model.predict(X_train)
y_test_pred = model.predict(X_test)

# Evaluación del modelo
from sklearn.metrics import accuracy_score, classification_report

train_acc = accuracy_score(y_train, y_train_pred)
test_acc = accuracy_score(y_test, y_test_pred)

# metricas en entrenamiento
f1_score_train = f1_score(y_train, y_train_pred, average='weighted')
precision_score_train = precision_score(y_train, y_train_pred, average='weighted')
recall_score_train = recall_score(y_train, y_train_pred, average='weighted')
# metricas en test
f1_score_test = f1_score(y_test, y_test_pred, average='weighted')
precision_score_test = precision_score(y_test, y_test_pred, average='weighted')
recall_score_test = recall_score(y_test, y_test_pred, average='weighted')

print(f"Train Accuracy: {train_acc:.4f}")
print(f"Test Accuracy: {test_acc:.4f}")

# Informe de clasificación
print("\nClassification Report - Train")
print(classification_report(y_train, y_train_pred))

print("\nClassification Report - Test")
print(classification_report(y_test, y_test_pred))

# Overfit gap
overfit_gap = train_acc - test_acc
print(f"\nOverfit Gap: {overfit_gap:.4f}")

Train Accuracy: 0.8047
Test Accuracy: 0.7305

Classification Report - Train
              precision    recall  f1-score   support

           0       0.88      0.75      0.81       263
           1       0.87      0.78      0.83       386
           2       0.75      0.90      0.82       529
           3       0.86      0.73      0.79       262
           4       0.75      0.87      0.80       261
           5       0.77      0.85      0.81       168
           6       0.89      0.74      0.81       129
           7       0.99      0.67      0.80       114
           9       0.30      0.47      0.36        17
          10       0.85      0.52      0.65        21

    accuracy                           0.80      2150
   macro avg       0.79      0.73      0.75      2150
weighted avg       0.82      0.80      0.81      2150


Classification Report - Test
              precision    recall  f1-score   support

           0       0.85      0.81      0.83        63
           1       0.74   

In [224]:
# columnas de X_train
X_train_columns = X_train.columns
X_train_columns

Index(['SyHeadOrientation', 'SxHeadOrientation', 'SzHeadOrientation',
       'SleftEyeClosed', 'SrightEyeClosed', 'SleftEyebrowLowered',
       'SleftEyebrowRaised', 'SrightEyebrowLowered', 'SrightEyebrowRaised',
       'SgazeDirectionForward',
       ...
       'ElbowRight_WristRightPlaneZXAxisXstdv',
       'ElbowRight_WristRightPlaneXYAxisYstdv',
       'ElbowRight_WristRightPlaneYZAxisZstdv',
       'WristRight_HandRightPlaneZXAxisXstdv',
       'WristRight_HandRightPlaneXYAxisYstdv',
       'WristRight_HandRightKinectZAxisstdv', 'Condition_N', 'Condition_T',
       'Blok_1', 'Blok_2'],
      dtype='object', length=136)

In [12]:
X_train_columns

Index(['SyHeadOrientation', 'SxHeadOrientation', 'SzHeadOrientation',
       'SmouthOpen', 'SleftEyeClosed', 'SleftEyebrowLowered',
       'SleftEyebrowRaised', 'SrightEyebrowLowered', 'SrightEyebrowRaised',
       'SgazeDirectionForward',
       ...
       'ElbowRight_WristRightPlaneXYAxisYstdv',
       'ElbowRight_WristRightPlaneYZAxisZstdv',
       'WristRight_HandRightPlaneZXAxisXstdv',
       'WristRight_HandRightPlaneXYAxisYstdv',
       'WristRight_HandRightKinectZAxisstdv', 'Condition_I', 'Condition_N',
       'Condition_T', 'Blok_2', 'Blok_3'],
      dtype='object', length=107)

In [225]:


# Guardar el modelo en un archivo .pkl
with open("modelo_DecisionTree.pkl", "wb") as f:
    pickle.dump(model, f)

# Guardar los parámetros y métricas en un archivo JSON
model_info = {
    "best_params": study.best_params,
    "train_metrics": {
        "accuracy": train_acc,
        "f1_score": f1_score_train,
        "precision": precision_score_train,
        "recall": recall_score_train
    },
    "test_metrics": {
        "accuracy": test_acc,
        "f1_score": f1_score_test,
        "precision": precision_score_test,
        "recall": recall_score_test
    },
    "overfit_gap": overfit_gap
}

with open("metricas_y_best_params_DecisionTree.json", "w") as f:
    json.dump(model_info, f, indent=4)

# Guardar las columnas de X_train en un archivo txt
with open("columnas_X_train_DecisionTree.txt", "w") as f:
    f.write("\n".join(X_train_columns))

print("\nModelo y parámetros guardados correctamente.")



Modelo y parámetros guardados correctamente.


Estos son recomendaciones de chatGPT, no hacerle caso

Aumentar el min_samples_leaf → Subirlo a 7-10 evitará que los árboles se vuelvan demasiado específicos.
Reducir max_depth a 6 o 7 → Un árbol menos profundo generaliza mejor.
Aumentar min_impurity_decrease → Subirlo a 0.05-0.07 para podar ramas irrelevantes.
Agregar Dropout o Regularización → Usar class_weight='balanced' puede ayudar si hay clases desbalanceadas.
Más datos de entrenamiento → Si es posible, aumentar el dataset con data augmentation o técnicas de síntesis de datos.
Ajustar max_samples → Disminuirlo a 0.5-0.55 para forzar más diversidad en los árboles.

In [226]:
df_resultados = pd.DataFrame({'y_test': y_test, 'y_pred': y_test_pred})

display(df_resultados)

,y_test,y_pred
2869,5,5
3126,0,0
1373,2,0
742,3,7
1603,1,1
...,...,...
488,4,4
2201,4,4
1458,7,7
2540,2,1
